<a href="https://colab.research.google.com/github/AlexKressner/WS25_Supply_Chain_Optimierung/blob/main/Spediteur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# pip als Paketmanager
! pip install -q pyscipopt

In [15]:
from pyscipopt import Model, quicksum

# Optimierungsmodell für Spediteur

In [16]:
# Erstellen einer Modellinstanz
scip = Model()

## Indexmengen

$i \in \mathcal{I}$, $j \in \mathcal{J}$

In [17]:
I = ['A1','A2','A3','A4','A5','A6'] # Menge der Aufträge
J = ['LKW1','LKW2','LKW3'] # Menge der Transportmittel

## Parameter (Daten)

$db_i$: Deckungsbeitrag

In [18]:
db={} # Deckungsbeiträge
db['A1']=10
db['A2']=15
db['A3']=25
db['A4']=5
db['A5']=11
db['A6']=8

$v_i$: Ladevolumen der Aufträge

In [19]:
# Auftragsvolumen (m^3)
v={}
v['A1']=3
v['A2']=4.5
v['A3']=9.25
v['A4']=1.2
v['A5']=4
v['A6']=2.7

$lv_j$: Ladevolumen LKW

In [20]:
# Ladevolumen LKW
lv={}
lv['LKW1']=10
lv['LKW2']=8
lv['LKW3']=5

## Entscheidungsvariablen

$X_{ij} \in \{0,1\}$

In [21]:
# Definition der Entscheidungsvariablen
x={}
for i in I:
  for j in J:
    x[i,j] = scip.addVar(vtype='B', name=f"X_{i}_{j}")
print('Entscheidungsvariablen =', scip.getVars())

Entscheidungsvariablen = [X_A1_LKW1, X_A1_LKW2, X_A1_LKW3, X_A2_LKW1, X_A2_LKW2, X_A2_LKW3, X_A3_LKW1, X_A3_LKW2, X_A3_LKW3, X_A4_LKW1, X_A4_LKW2, X_A4_LKW3, X_A5_LKW1, X_A5_LKW2, X_A5_LKW3, X_A6_LKW1, X_A6_LKW2, X_A6_LKW3]


## Zielfunktion

$\text{max} DB = \sum_{i,j} db_i \cdot X_{i,j}$

In [22]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(db[i]*x[i,j] for i in I for j in J), sense="maximize")

## Nebenbedingungen/ Restriktionen

$\sum_i v_i \cdot X_{i,j} \leq lv_j$ für jedes $j \in \mathcal{J}$

In [23]:
for j in J:
  scip.addCons(quicksum(v[i]*x[i,j] for i in I)<=lv[j], name=f"Laderaumvolumen_LKW_{j}")

$\sum_j X_{i,j} \leq 1$ für alle $i \in \mathcal{I}$

In [24]:
for i in I:
  scip.addCons(quicksum(x[i,j] for j in J)<=1, name=f"Einmaliges_Verladen_Auftrag_{i}")

In [25]:
print('Nebenbedingungen =', scip.getConss())

Nebenbedingungen = [Laderaumvolumen_LKW_LKW1, Laderaumvolumen_LKW_LKW2, Laderaumvolumen_LKW_LKW3, Einmaliges_Verladen_Auftrag_A1, Einmaliges_Verladen_Auftrag_A2, Einmaliges_Verladen_Auftrag_A3, Einmaliges_Verladen_Auftrag_A4, Einmaliges_Verladen_Auftrag_A5, Einmaliges_Verladen_Auftrag_A6]


## Berechnung der Lösung

In [26]:
scip.optimize()
# Status des Solvers
status = scip.getStatus()
print(f"Status des Solvers: {status} \n")

if status == "optimal":
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', round(scip.getObjVal()))
    for j in J:
      print(f'\n{j}:')
      for i in I:
        print(f'X_{i}_{j} =', round(scip.getVal(x[i,j])))
else:
    print('Problem hat keine Lösung')

Status des Solvers: optimal 

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 64

LKW1:
X_A1_LKW1 = 0
X_A2_LKW1 = 0
X_A3_LKW1 = 1
X_A4_LKW1 = 0
X_A5_LKW1 = 0
X_A6_LKW1 = 0

LKW2:
X_A1_LKW2 = 0
X_A2_LKW2 = 0
X_A3_LKW2 = 0
X_A4_LKW2 = 1
X_A5_LKW2 = 1
X_A6_LKW2 = 1

LKW3:
X_A1_LKW3 = 0
X_A2_LKW3 = 1
X_A3_LKW3 = 0
X_A4_LKW3 = 0
X_A5_LKW3 = 0
X_A6_LKW3 = 0
